In [ ]:
!pip install fuzzywuzzy==0.6.1

import pandas as pd
import re
from collections import Counter, defaultdict
import numpy as np
from fuzzywuzzy import fuzz
import os

# Cargar el archivo Excel
file_path = "BibliometrixExportFile20250220.xlsx"
df = pd.read_excel(file_path)

# Asegurar que no haya valores NaN en las columnas de interés
df['FU'] = df['FU'].fillna("")
df['FX'] = df['FX'].fillna("")
df['TI'] = df['TI'].fillna("Sin título")

# Lista de términos genéricos que NO son instituciones reales
stopwords = set([
    'FUNDING SOURCE', 'FUNDING SOURC', 'GRANT', 'GRANTS', 'WAS  THE ',
    'SUPPORTED BY', 'FUNDED BY', 'FINANCED BY', 'AND',
    'THROUGH', 'PROJECT', 'PROGRAM', 'CODE', 'NUMBER', 'THIS WORK', 'WITHIN THE SCOPE OF',
    'THE AUTHORS', 'AUTHOR', 'ACKNOWLEDGE', 'ACKNOWLEDGES', 'FUND',
    'FOUNDATION', 'SUPPORTED', 'SUPPORT', 'FINANCIAL', 'UNDER', "INTERPRETATION OF DAT",
    'ANY OPINIONS', 'ANALYSI', 'IS PART OF THE RESEARCH', 'PROGRA'
])

# Expresiones para normalizar nombres de agencias
agency_aliases = {
    'NATIONAL SCIENCE FOUNDATION': ['NSF', 'NATIONAL SCIENCE', 'NATL SCIENCE FOUNDATION'],
    'EUROPEAN COMMISSION': ['EC', 'EUROPEAN UNION', 'EU PROJECT', 'HORIZON 2020', 'HORIZON EUROPE'],
    'NATIONAL INSTITUTES OF HEALTH': ['NIH', 'NATL INSTITUTES HEALTH', 'NATL INST HEALTH'],
    'NATIONAL NATURAL SCIENCE FOUNDATION OF CHINA': ['NSFC', 'NAT NAT SCI FOUND CHINA', 'NATL NAT SCI FOUND CHINA']
}

# Patrones para extraer nombres de instituciones más robustos
patterns = [
    r'\b(?:funded by|supported by|granted by|financed by)\s+([A-Z][A-Z\s\-&\.]+)',  # Ej: funded by XYZ
    r'\b([A-Z][A-Z\s\-&\.]+)(?:\s+grant|\s+project|\s+contract)(?:\s+\#|\s+no\.|\s+number)?(?:\s+[A-Z0-9\-\/\.]+)?', # Ej: XYZ GRANT #1234
    r'\b([A-Z][A-Z\s\-&\.]+)\s*\[?[A-Z0-9\-\/]+\]?',  # Ej: XYZ [1234]
    r'\b([A-Z][A-Z\s\-&\.]+)(?:,)?\s*(?:PORTUGAL|BRAZIL|PERU|USA|CANADA|SPAIN|CHINA|UK|GERMANY|FRANCE|JAPAN)?',  # Ej: XYZ, BRAZIL
    r'(?:GRANT|CONTRACT|AWARD)\s+(?:FROM|BY)\s+([A-Z][A-Z\s\-&\.]+)', # Ej: GRANT FROM XYZ
]

# Función para normalizar nombres de agencias
def normalize_agency_name(name):
    # Eliminar palabras comunes que no aportan valor distintivo
    for word in stopwords:
        name = re.sub(r'\b' + re.escape(word) + r'\b', '', name)

    # Normalizar espacios
    name = re.sub(r'\s+', ' ', name).strip()

    # Verificar si es un alias conocido
    for standard_name, aliases in agency_aliases.items():
        if any(alias in name for alias in aliases):
            return standard_name

    return name

# Función para extraer y limpiar instituciones desde un texto
def extract_funding_sources(text):
    if not isinstance(text, str):
        return []

    items = [part.strip() for part in re.split(r'[;,]', text.upper()) if part.strip()]
    results = []

    for item in items:
        # Aplicar patrones de búsqueda
        candidates = []
        for pattern in patterns:
            matches = re.findall(pattern, item)
            candidates.extend(matches)

        # Limpiar y normalizar los candidatos
        for candidate in candidates:
            cleaned = candidate.strip()
            if cleaned and len(cleaned) > 3:
                normalized = normalize_agency_name(cleaned)
                if normalized and len(normalized) > 3:
                    results.append(normalized)

    # Eliminar duplicados manteniendo el orden
    unique_results = []
    for result in results:
        if result not in unique_results:
            unique_results.append(result)

    return unique_results

# Función para agrupar nombres similares de agencias
def group_similar_agencies(agencies_dict, similarity_threshold=80):
    agencies = list(agencies_dict.keys())
    grouped_agencies = {}
    processed = set()

    for i, agency1 in enumerate(agencies):
        if agency1 in processed:
            continue

        group = [agency1]
        for j in range(i+1, len(agencies)):
            agency2 = agencies[j]
            if agency2 in processed:
                continue

            # Comparar similitud usando fuzzywuzzy
            similarity = fuzz.ratio(agency1, agency2)
            if similarity >= similarity_threshold:
                group.append(agency2)
                processed.add(agency2)

        # Usar la agencia con más artículos como nombre del grupo
        main_agency = max(group, key=lambda x: len(agencies_dict[x]))
        grouped_agencies[main_agency] = []

        # Combinar artículos de todas las agencias del grupo
        for agency in group:
            grouped_agencies[main_agency].extend(agencies_dict[agency])
            processed.add(agency)

        # Eliminar duplicados
        grouped_agencies[main_agency] = list(set(grouped_agencies[main_agency]))

    return grouped_agencies

# Función principal para procesar el archivo y agrupar resultados
def analyze_funding_sources(df, output_dir="resultados_funding"):
    # Crear directorio para resultados si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Diccionario para almacenar los artículos por cada organismo de funding
    funder_to_articles = defaultdict(list)

    # Procesar cada artículo
    for index, row in df.iterrows():
        # Extraer instituciones de funding de este artículo
        article_funders = extract_funding_sources(row['FU']) + extract_funding_sources(row['FX'])

        # Eliminar duplicados
        article_funders = list(set(article_funders))

        # Si el artículo tiene un título, usarlo; de lo contrario usar el índice
        article_id = row['TI'] if pd.notna(row['TI']) else f"Article_{index}"

        # Asociar este artículo con cada uno de sus organismos de funding
        for funder in article_funders:
            if article_id not in funder_to_articles[funder]:
                funder_to_articles[funder].append(article_id)

    # Agrupar agencias con nombres similares
    grouped_funders = group_similar_agencies(funder_to_articles)

    # Crear DataFrame con los resultados agrupados
    results = []
    for funder, articles in grouped_funders.items():
        results.append({
            "Funder": funder,
            "Count": len(articles),
            "Articles": articles
        })

    funding_df = pd.DataFrame(results)
    funding_df = funding_df.sort_values(by="Count", ascending=False).reset_index(drop=True)

    # Guardar resultados
    funding_df.to_excel(f"{output_dir}/instituciones_financiadoras_agrupadas.xlsx", index=False)

    # Crear archivo separado para cada financiador principal
    top_funders = funding_df.head(20)
    for _, row in top_funders.iterrows():
        funder_name = row['Funder']
        articles = row['Articles']

        # Crear un DataFrame con los artículos de este financiador
        articles_df = pd.DataFrame({
            "Article_Title": articles,
            "Funder": funder_name
        })

        # Nombre de archivo seguro (sin caracteres problemáticos)
        safe_name = re.sub(r'[^\w\s-]', '', funder_name)
        safe_name = re.sub(r'[\s-]+', '_', safe_name).lower()

        # Guardar en archivo Excel
        articles_df.to_excel(f"{output_dir}/({str(articles_df.shape[0])}){safe_name}_articles.xlsx", index=False)

    return funding_df

# Ejecutar el análisis
print("Iniciando análisis de fuentes de financiación...")
result_df = analyze_funding_sources(df)

# Mostrar resultados
print("\nPrincipales organismos de financiación:")
print(result_df[['Funder', 'Count']].head(20))

# Mostrar artículos de los 20 principales financiadores
for i, row in result_df.head(20).iterrows():
    funder = row['Funder']
    count = row['Count']
    articles = row['Articles']

    print(f"\nArtículos financiados por {funder} (Total: {count}):")
    for j, article in enumerate(articles, 1):
        print(f"{j}. {article}")

  Preparing metadata (setup.py) ... done
  Created wheel for fuzzywuzzy: filename=fuzzywuzzy-0.6.1-py3-none-any.whl size=11999 sha256=b212970d25bd1ef639a0f2c5716014b5b585db199eb94e6ea3b40cdc57451204
  Stored in directory: /root/.cache/pip/wheels/97/25/46/fa37a3442870c6ee6b38e9f4dc209011fa3883337d1a51ba82
Successfully built fuzzywuzzy


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Iniciando análisis de fuentes de financiación...

Principales organismos de financiación:
                                               Funder  Count
0                                 EUROPEAN COMMISSION     68
1                         NATIONAL SCIENCE FOUNDATION     13
2                       NATIONAL INSTITUTES OF HEALTH      6
3                                            ANALYSIS      4
4                                  ACADEMY OF FINLAND      3
5                                                CNS-      3
6                                         ANY OPINION      3
7                GERMAN MINISTRY OF EDUCATION RESEARC      3
8                              INTERPRETATION OF DATA      3
9   DEVELOPED BY HCI-DUXAIT RESEARCH GROUP. HCI-DU...      3
10  VIRTUALIZACION DEL PROCESO DE EVALUACION DE EX...      3
11  VIRTUALIZATION OF THE USER EXPERIENCE EVALUATI...      3
12            PONTIFICIA UNIVERSIDAD CATOLICA DEL PER      3
13                                          S HORIZON   